In [2]:
import pandas as pd # package for high-performance, easy-to-use data structures and data analysis
import numpy as np # fundamental package for scientific computing with Python
import matplotlib
import matplotlib.pyplot as plt # for plotting
import seaborn as sns # for making plots with seaborn
color = sns.color_palette()
from scipy.stats import kurtosis
from numpy import array
from matplotlib import cm
# Supress unnecessary warnings so that presentation looks clean
import warnings
warnings.filterwarnings("ignore")

from sklearn.externals import joblib
import datetime
import time
# Print all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [19]:
train = joblib.load('train1.pkl')    # 载入数据

In [57]:
listing_info = joblib.load('listing_info1.pkl')    # 载入数据

In [20]:
user_repay_logs = joblib.load('user_repay_logs1.pkl')    # 载入数据

In [ ]:
user_behavior_logs = joblib.load('user_behavior_logs1.pkl')    # 载入数据

In [21]:
user_info = joblib.load('user_info1.pkl')    # 载入数据

In [22]:
user_taglist = joblib.load('user_taglist1.pkl')    # 载入数据

In [23]:
test = joblib.load('test1.pkl')    # 载入数据

In [18]:
# 将test中有历史交易记录的user对应记录提取出来作为train1
train_user_id=set(train['user_id'].tolist())
test_user_id=set(test['user_id'].tolist())
repay_logs_user_id = set(user_repay_logs['user_id'].tolist())
repay_logs_listing_id = set(user_repay_logs['listing_id'].tolist())
train_listing_id=set(train['listing_id'].tolist())

NameError: name 'train' is not defined

In [15]:
x1 = test_user_id - train_user_id
user_no = x1 - repay_logs_user_id    # test中没在train、repay_log中出现过的user
user_no_num = test[test['user_id'].isin(user_no)]       # 保存test中从未出现在train、repay_log中的部分
user_yes_num = test[~test['user_id'].isin(user_no)]     # 保存test中在train、repay_log中出现过的部

In [232]:
user_yes_train = train[train['user_id'].isin(test_user_id&train_user_id)]   
# test中在train出现过的user

print(user_yes_train.shape)

(33379, 7)


In [20]:
user_yes_train.head()

,user_id,listing_id,auditing_date,due_date,due_amt,repay_date,repay_amt
52,399026,5105859,2018-12-12,2019-01-12,396.0697,2019-01-12,396.0697
129,491230,4773399,2018-10-28,2018-11-28,943.9129,2018-11-23,943.9129
148,889013,4916211,2018-11-15,2018-12-15,170.1841,2018-12-15,170.1841
153,493599,2393635,2018-02-23,2018-03-23,390.6251,2018-03-22,390.6251
157,741140,4460746,2018-09-17,2018-10-17,394.6893,2018-10-16,394.6893


In [231]:
user_yes_log = user_repay_logs[user_repay_logs['user_id'].isin(test_user_id&repay_logs_user_id)]   
# test中在log出现过的user

print(user_yes_log.shape)

(2746766, 7)


In [15]:
def logpart(x): 
    # 处理日志数据
    x['due_date'] = pd.to_datetime(x.due_date,format = '%Y-%m-%d')
    x['repay_date'] = pd.to_datetime(x.repay_date,format = '%Y-%m-%d')
    x['month'] = x['due_date'].map(lambda x:x.month)
    x['year'] = x['due_date'].map(lambda x:x.year)
    x['day'] = x['due_date'].map(lambda x:x.day)
    x['delta'] = x['month'] - x['order_id']
    x.loc[x['delta'] <= 0,'month'] = x.loc[x['delta'] <= 0,'delta'].map(lambda x: x+12) 
    x.loc[x['delta'] <= 0,'year'] = x.loc[x['delta'] <= 0,'year'].map(lambda x: x-1)
    x.loc[x['delta'] > 0,'month']= x.loc[x['delta'] > 0,'delta']
    x.loc[x['delta'] > 0,'year'] = x.loc[x['delta'] > 0,'year']
    x['year'] =  [int(i) for i in x['year']]
    x['year'] = x['year'].astype(str)
    x['month'] =  [int(i) for i in x['month']]
    x['month'] = x['month'].astype(str)
    x['day'] = x['day'].astype(str)
    x['auditing_date'] = x['year'].str.cat([x['month'],x['day']],sep='-')
    x['auditing_date'] = pd.to_datetime(x.auditing_date,format = '%Y-%m-%d')
    x.drop(['order_id','month','year','day','delta'],axis=1,inplace=True)
    order = ['user_id','listing_id','auditing_date','due_date','due_amt','repay_date','repay_amt']
    x = x[order]
    return x     # 日期数据都转成了日期格式

In [16]:
def trainpart(x):
    # 处理train数据
    x.loc[x['repay_date'] == '\\N','repay_date']= '2200-01-01'
    x.loc[x['repay_amt'] == '\\N','repay_amt'] = x.loc[x['repay_amt'] == '\\N','due_amt']
    x['due_date'] = pd.to_datetime(x.due_date,format = '%Y-%m-%d')
    x['repay_date'] = pd.to_datetime(x.repay_date,format = '%Y-%m-%d')
    x['auditing_date'] = pd.to_datetime(x.auditing_date,format = '%Y-%m-%d')
    return x

In [13]:
def testpart(x):
    # 处理测试集数据
    x['due_date']= pd.to_datetime(x.due_date,format = '%Y-%m-%d')
    x['auditing_date'] = pd.to_datetime(x.auditing_date,format = '%Y-%m-%d')
    return x

In [17]:
user_yes_train1 = trainpart(user_yes_train)
user_yes_log1 = logpart(user_yes_log)

NameError: name 'user_yes_train' is not defined

In [24]:
test1 = testpart(test)

In [225]:
user_yes_log.head()

,user_id,listing_id,auditing_date,due_date,due_amt,repay_amt
repay_date,,,,,,
2016-10-01,270705,6256,2016-09-28,2017-07-28,245.2538,245.2538
2016-10-01,270705,6256,2016-09-28,2017-09-28,245.2538,245.2538
2016-10-01,270705,6256,2016-09-28,2017-08-28,245.2538,245.2538
2016-10-17,282185,10198,2016-10-17,2017-10-17,159.4149,159.4149
2016-10-17,282185,10198,2016-10-17,2017-09-17,159.4149,159.4149


In [224]:
user_yes_log2 = user_yes_log1.sort_values(by=["repay_date"]).set_index(["repay_date"])

In [141]:
log1 = user_yes_log2.loc['2200-01-01'].reset_index()
log2 = log1.sort_values(by=["due_date"]).set_index(["due_date"])
log3 = log2.truncate(after="2019-03-01").reset_index()     # 逾期部分可用数据范围

In [147]:
log3.shape

(125689, 7)

In [152]:
user_yes_log1["repay_date"] = user_yes_log1["repay_date"].astype(str)

log4 = user_yes_log1.loc[user_yes_log1["repay_date"]!='2200-01-01']
log4["repay_date"] = pd.to_datetime(log4.repay_date,format = '%Y-%m-%d')

In [153]:
log5 = log4.sort_values(by=["repay_date"]).set_index(["repay_date"])
log6 = log5.truncate(after="2019-03-01").reset_index()     # 未逾期部分可用数据范围

In [154]:
log6.shape

(2484832, 7)

In [168]:
order = ['user_id','listing_id','auditing_date','due_date','due_amt','repay_date','repay_amt']
log3 = log3[order]
log6 = log6[order]

In [169]:
log3.head()

,user_id,listing_id,auditing_date,due_date,due_amt,repay_date,repay_amt
0,34612,271495,2017-05-09,2017-07-09,1065.9782,2200-01-01,1065.9782
1,34612,8272,2016-10-09,2017-07-09,303.0636,2200-01-01,303.0636
2,153514,20854,2016-11-11,2017-07-11,492.2594,2200-01-01,492.2594
3,260235,403888,2017-06-12,2017-07-12,295.6113,2200-01-01,295.6113
4,155973,69240,2017-01-13,2017-07-13,386.1742,2200-01-01,386.1742


In [171]:
total = pd.concat([log3,log6,user_yes_train1],axis=0,ignore_index=True)
print(total.shape)

(2643900, 7)


In [172]:
total.drop_duplicates(keep='first',inplace=True)
print(total.shape)     # 删除重复的记录

(2641903, 7)


In [173]:
len(total['user_id'].unique())

119503

In [174]:
joblib.dump(total, 'total1.pkl')    # 存储合并的训练集

['total1.pkl']

In [32]:
total = joblib.load('total1.pkl')    # 载入数据

In [33]:
total['days']=(total['due_date']-total['repay_date']).apply(lambda x:x.days)   
# 提前还款天数（标签）

In [34]:
total['days']=total['days'].apply(lambda x: x if x>=0 else -1 )   # 逾期的标为-1

In [241]:
total.head()

,user_id,listing_id,auditing_date,due_date,due_amt,repay_date,repay_amt,days
0,34612,271495,2017-05-09,2017-07-09,1065.9782,2200-01-01,1065.98,-1
1,34612,8272,2016-10-09,2017-07-09,303.0636,2200-01-01,303.064,-1
2,153514,20854,2016-11-11,2017-07-11,492.2594,2200-01-01,492.259,-1
3,260235,403888,2017-06-12,2017-07-12,295.6113,2200-01-01,295.611,-1
4,155973,69240,2017-01-13,2017-07-13,386.1742,2200-01-01,386.174,-1


In [43]:
sp_total = total[total['days']<32]

In [36]:
total_1 = total.copy()   # 备份合并后的训练集

In [44]:
total_2 = sp_total.copy()   

In [38]:
sp_total.head()

,user_id,listing_id,auditing_date,due_date,due_amt,repay_date,repay_amt,days
0,34612,271495,2017-05-09,2017-07-09,1065.9782,2200-01-01,1065.98,-1
1,34612,8272,2016-10-09,2017-07-09,303.0636,2200-01-01,303.064,-1
2,153514,20854,2016-11-11,2017-07-11,492.2594,2200-01-01,492.259,-1
3,260235,403888,2017-06-12,2017-07-12,295.6113,2200-01-01,295.611,-1
4,155973,69240,2017-01-13,2017-07-13,386.1742,2200-01-01,386.174,-1


In [255]:
total_2[total_2['days']!=-1].shape

(1912611, 8)

In [45]:
y_label = sp_total['days']

In [90]:
sp_total.head()

,user_id,listing_id,auditing_date,due_date,due_amt,repay_date,repay_amt,days
0,34612,271495,2017-05-09,2017-07-09,1065.9782,2200-01-01,1065.98,-1
1,34612,8272,2016-10-09,2017-07-09,303.0636,2200-01-01,303.064,-1
2,153514,20854,2016-11-11,2017-07-11,492.2594,2200-01-01,492.259,-1
3,260235,403888,2017-06-12,2017-07-12,295.6113,2200-01-01,295.611,-1
4,155973,69240,2017-01-13,2017-07-13,386.1742,2200-01-01,386.174,-1


In [48]:
del total_2['repay_amt'],total_2['repay_date'],total_2['days']

In [99]:
testtotal0 = pd.concat([total_2,test1],axis=0,ignore_index=True)
print(testtotal0.shape)

(2168327, 5)


In [96]:
total_2.head()

,user_id,listing_id,auditing_date,due_date,due_amt
0,34612,271495,2017-05-09,2017-07-09,1065.9782
1,34612,8272,2016-10-09,2017-07-09,303.0636
2,153514,20854,2016-11-11,2017-07-11,492.2594
3,260235,403888,2017-06-12,2017-07-12,295.6113
4,155973,69240,2017-01-13,2017-07-13,386.1742


In [97]:
test1.head()

,user_id,listing_id,auditing_date,due_date,due_amt
0,498765,5431438,2019-03-12,2019-04-12,138.5903
1,34524,5443211,2019-03-15,2019-04-15,208.0805
2,821741,5461707,2019-03-22,2019-04-22,421.2097
3,263534,5472320,2019-03-26,2019-04-26,212.6537
4,238853,5459750,2019-03-21,2019-04-21,817.4593


In [52]:
testtotal0['year'] = testtotal0['due_date'].dt.year.astype(str)
testtotal0['month'] = testtotal0['due_date'].dt.month.astype(str)
testtotal0['day'] = testtotal0['due_date'].dt.day.astype(str)
testtotal0['dayofweek'] = testtotal0['due_date'].dt.dayofweek.astype(str)

In [53]:
testtotal0['days1']=(testtotal0['due_date']-testtotal0['auditing_date']).apply(lambda x:x.days)
# 应还款日期与交易日间隔天数

In [54]:
testtotal1 = pd.get_dummies(testtotal0)

In [69]:
testtotal2 = pd.merge(testtotal1,listing_info,on=['user_id','listing_id'],how='left')   # 合并标的资产信息

In [75]:
testtotal2.drop(['auditing_date_y'],inplace=True,axis=1)
testtotal2.rename(columns={'auditing_date_x': 'auditing_date'},inplace=True)

In [76]:
testtotal2.head()

,user_id,listing_id,auditing_date,due_date,due_amt,days1,year_2017,year_2018,year_2019,month_1,month_10,month_11,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,day_1,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_2,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_3,day_30,day_31,day_4,day_5,day_6,day_7,day_8,day_9,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,term,rate,principal
0,34612,271495,2017-05-09,2017-07-09,1065.9782,61,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,12,9.3,12170
1,34612,8272,2016-10-09,2017-07-09,303.0636,273,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,12,9.3,3460
2,153514,20854,2016-11-11,2017-07-11,492.2594,242,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,12,9.3,5620
3,260235,403888,2017-06-12,2017-07-12,295.6113,30,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,6,8.6,1730
4,155973,69240,2017-01-13,2017-07-13,386.1742,181,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,6,8.6,2260


In [77]:
trainnew = testtotal2[:len(total_2)]
testnew = testtotal2[len(total_2):]    # 得到一起处理时间、标的资产特征的训练集、测试集

In [256]:
group = total_2.groupby('user_id', as_index=False)

In [273]:
total_3 = total_2.merge(
    group['days'].agg({
        'days_max': 'max', 'days_median': 'median',
        'days_mean': 'mean', 'days_sum':'sum','days_std': 'std', 'days_skew': 'skew', 'days_kurt':kurtosis
    }), on='user_id', how='left'
)    # 还款天数的统计量

In [274]:
total_4 = total_3.merge(
    group['due_amt'].agg({
        'due_amt_max': 'max', 'due_amt_min': 'min', 'due_amt_median': 'median',
        'due_amt_mean': 'mean', 'due_amt_sum': 'sum', 'due_amt_std': 'std',
        'due_amt_skew': 'skew', 'due_amt_kurt': kurtosis, 'due_amt_ptp': np.ptp
    }), on='user_id', how='left'
)     # 还款金额的统计量

In [276]:
label = total_4['days']

In [277]:
total_5 = total_4.copy()
total_6 = total_5.copy()

In [5]:
total_5.drop(['listing_id','auditing_date','due_date','due_amt','due_date','repay_date','repay_amt','days'],inplace=True,axis=1)

In [6]:
total_5.drop_duplicates(keep='first',inplace=True)

In [81]:
total_5.shape

(118125, 17)

In [79]:
test2 = testnew.merge(total_5,on='user_id',how='left')   # 测试集

In [82]:
train2 = trainnew.merge(total_5,on='user_id',how='left')

In [106]:
train3 = pd.concat([train2,y_label],axis=1)   # 训练集，label为days

In [108]:
train3.head()

,user_id,listing_id,auditing_date,due_date,due_amt,days1,year_2017,year_2018,year_2019,month_1,month_10,month_11,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,day_1,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_2,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_3,day_30,day_31,day_4,day_5,day_6,day_7,day_8,day_9,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,term,rate,principal,days_max,days_median,days_mean,days_sum,days_std,days_skew,days_kurt,due_amt_max,due_amt_min,due_amt_median,due_amt_mean,due_amt_sum,due_amt_std,due_amt_skew,due_amt_kurt,due_amt_ptp,days
0,34612.0,271495.0,2017-05-09,2017-07-09,1065.9782,61.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,12.0,9.3,12170.0,29.0,1.0,3.636986,531.0,5.605269,2.487345,7.136847,1642.3783,127.3298,361.7494,432.723740,63177.6661,289.092940,1.880547,3.726120,1515.0485,-1.0
1,34612.0,8272.0,2016-10-09,2017-07-09,303.0636,273.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,12.0,9.3,3460.0,29.0,1.0,3.636986,531.0,5.605269,2.487345,7.136847,1642.3783,127.3298,361.7494,432.723740,63177.6661,289.092940,1.880547,3.726120,1515.0485,-1.0
2,153514.0,20854.0,2016-11-11,2017-07-11,492.2594,242.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,12.0,9.3,5620.0,31.0,0.0,2.432836,163.0,6.025763,3.476942,12.890171,634.1695,99.5897,192.3113,317.939231,21301.9285,216.224881,0.689874,-1.426035,534.5798,-1.0
3,260235.0,403888.0,2017-06-12,2017-07-12,295.6113,30.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,8.6,1730.0,30.0,0.0,3.508621,407.0,7.423225,2.307174,4.684009,1541.6573,152.4077,405.8958,505.029433,58583.4142,346.161894,1.962962,2.940583,1389.2496,-1.0
4,155973.0,69240.0,2017-01-13,2017-07-13,386.1742,181.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0,8.6,2260.0,27.0,7.5,8.955556,806.0,9.294085,0.386569,-1.378387,692.8420,97.3003,239.2444,266.352062,23971.6856,170.853994,1.471807,1.268337,595.5417,-1.0


In [109]:
test2.head()

,user_id,listing_id,auditing_date,due_date,due_amt,days1,year_2017,year_2018,year_2019,month_1,month_10,month_11,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,day_1,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_2,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_3,day_30,day_31,day_4,day_5,day_6,day_7,day_8,day_9,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,term,rate,principal,days_max,days_median,days_mean,days_sum,days_std,days_skew,days_kurt,due_amt_max,due_amt_min,due_amt_median,due_amt_mean,due_amt_sum,due_amt_std,due_amt_skew,due_amt_kurt,due_amt_ptp
0,498765,5431438,2019-03-12,2019-04-12,138.5903,31,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,12,7.2,1600,31.0,0.0,2.869565,132.0,8.486875,2.882447,6.002045,560.9080,191.2131,307.30710,310.071128,14263.2719,79.445406,1.363017,2.299467,369.6949
1,34524,5443211,2019-03-15,2019-04-15,208.0805,31,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,9,6.9,1820,0.0,0.0,-0.142857,-1.0,0.377964,-2.645751,2.166667,478.0331,439.5687,478.03310,472.538186,3307.7673,14.538177,-2.645751,2.166667,38.4644
2,821741,5461707,2019-03-22,2019-04-22,421.2097,31,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,6,6.5,2480,25.0,5.5,7.666667,138.0,9.042384,0.907477,-0.713752,963.6356,107.1232,389.60415,460.264533,8284.7616,287.975922,1.085262,-0.513161,856.5124
3,263534,5472320,2019-03-26,2019-04-26,212.6537,31,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,9,6.9,1860,31.0,2.0,6.043478,139.0,9.107830,1.715661,1.261076,1690.1890,317.9689,1135.32890,917.817357,21109.7992,469.118757,0.035617,-1.647867,1372.2201
4,238853,5459750,2019-03-21,2019-04-21,817.4593,31,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,9,6.9,7150,9.0,3.0,3.666667,11.0,5.033223,0.585583,-1.500000,2955.0953,111.2401,2955.09530,2007.143567,6021.4307,1641.900565,-1.732051,-1.500000,2843.8552


In [110]:
joblib.dump(train3, 'train3.pkl')    # 训练集
joblib.dump(test2, 'test2.pkl')    # 训练集

['test2.pkl']